In [ ]:
%%HTML
<style>
.container { width:100% }
</style>

In [ ]:
%run Util/00_imports.ipynb

# Einleitung

Bei Schach handelt es sich um ein Brettspiel, das insgesamt von zwei Spielern gespielt werden kann.

## Aufgabenstellung

Das Ziel dieser Arbeit besteht in der Implementierung einer künstlichen Intelligenz, die in der Lage ist, für einfache
Spielsituationen den optimalen Zug und daraufhin den optimalen Spielverlauf anzugeben. Für die Implementierung wird die
Programmiersprache Python verwendet. Mithilfe der Bibliothek python-chess und einem Jupyter-Notebook soll es möglich
sein, die momentane Situation des Spiels einzutragen und für folgende Situationen einen Vorschlag für den nächsten Zug
zu erhalten:
1. König und Turm gegen König
2. König und zwei Läufer gegen König
3. König, Läufer und Springer gegen König
4. König und Bauer gegen König

# Schach

## Spielbrett und Spielfiguren
Ein Schachbrett besteht aus insgesamt 64 Feldern, die in einer 8x8 Matrix angeordnet sind. Dabei werden die Spalten mit
den Buchstaben a-h und die Zeilen mit den Zahlen 1-8 beschriftet. Das typische Schachmuster entsteht durch einen Wechsel
der weißen und schwarzen bzw dunklen und hellen Felder. Dementsprechend sind insgesamt 32 weiße und 32 schwarze Felder
auf dem Schachbrett zu finden. Die Felder können mithilfe der zuvor genannten Beschriftung auch eindeutig identifiziert
werden. Sofern in dem folgenden Dokument Felder durch beispielsweise "c5" angesprochen werden, ist hiermit das Feld
gemeint, das in der Spalte "C" und der Zeile "5" zu finden ist.

Im Folgenden werden die Spalten des Schachbretts als Linien und die Zeilen als Reihen bezeichnet.

Dieses Schachbrett wird grundsätzlich von 32 Figuren besetzt (16 weiße und 16 schwarze Figuren). Eine Seite besteht aus
insgesamt sechs unterschiedlichen Figuren. Bei diesen Figuren handelt es sich um:

- Bauern (8)
- Springer (2)
- Läufer (2)
- Türme (2)
- Dame (1)
- König (1)

Die Zahl in den Klammern steht hierbei für die Anzahl an Figuren pro Seite. Die Aufstellung zu Beginn des Spiels kann
aus der folgenden Zelle entnommen werden:

In [ ]:
board = chess.Board()
board

#### Bewegungsmöglichkeiten der Figuren
Jeder der bereits genannten Figuren besitzt einerseits einen eigenen Wert, aber auch ein eigenes Bewegungsmuster.
Diese lauten wie folgt:
- **König** (unendlich): Der König gehört mit zu den unbeweglichsten Figuren auf dem Spielfeld. Er kann pro Zug nur ein Feld entlang
einer Reihe, Linie oder Diagonalen gehen. Jedoch besitzt dieser die Möglichkeit in jede Richtung eine gegnerische Figur
zu schlagen. Weiterhin gilt bei dem Ziehen mit dem König zu beachten, dass das angestrebte Feld nicht durch eine
gegnerische Figur abgedeckt wird. Sofern dies der Fall ist, darf der König dieses Feld nicht betreten.
- **Türme** (5): Der Turm besitzt die Möglichkeit entlang einer Reihe oder Linie so viele Felder zu überqueren wie man
möchte (maximal bis zum Ende des Spielfeldes. Dabei gilt es aber zu beachten, dass er durch seine eigenen Figuren
blockiert wird. Gegnerische Figuren kann er jedoch schlagen.
- **Läufer** (3): Der Läufer ähnelt dem Bewegungsmuster eines Turmes mit dem Unterschied, dass dieser sich nur über die
Diagonalen bewegen kann. Er kann jedoch selbst von den eigenen Figuren blockiert werden, aber auch gegnerische Figuren
schlagen.
- **Dame** (9): Die Dame zählt zu den beweglichsten Figuren auf dem Spielfeld. Sie kombiniert die Bewegungsmuster des
Läufers und des Turms. Sie kann dennoch von eigenen Figuren ebenfalls blockiert werden, gleichermaßen aber auch
gegnerische Figuren schlagen.
- **Bauer** (1): Der Bauer ist die erste Figur, die ein ungewöhnlicheres Bewegungsmuster besitzt. Dieser kann nur,
sofern er noch nicht bewegt wurde, zwei Schritte entlang der Spalte nach vorne machen. Ansonsten kann er immer nur einen
Schritt nach vorne machen. Die Besonderheit, die mit dem Bauern eingeht, liegt in der Richtung, in die ein Bauer
schlagen darf. Dieser darf nämlich nur diagonal nach vorne schlagen. Weiterhin verwandelt sich der Bauer in eine
beliebige Spielfigur (außer dem Bauern selbst und einem zweiten König) sobald er die Grundlinie des Gegners erreicht hat.
- **Springer** (3): Der Springer besitzt im Gegensatz zu den bereits beschriebenen Figuren keine lineare Bewegungsrichtung.
Diese spiegelt sich in der Bewegung um zwei Felder nach vorne und einem Feld zur Seite wider. Dieses Verfahren gilt in
jede Richtung, sodass der Springer im Optimalfall acht Felder erreichen kann. Der Name des Springers kommt dadurch
zustande, dass er die einzige Figur ist, die andere Figuren überspringen darf. Nur das "Zielfeld" kann durch eine eigene
Figur blockiert werden. Auf diesem Feld kann aber genauso eine gegnerische Figur stehen, die durch den Springer
geschlagen werden kann.

Weiterhin besitzen die Figuren auch Werte, die die Bedeutung der Figuren im Verhältnis zu den anderen aufzeigt. Diese
werden in Form der Werte in den Klammern gekennzeichnet.

## Spielablauf

In einem Spiel ziehen die Spieler immer abwechselnd eine Figur ihrer Farbe. Den ersten Zug hat dabei immer weiß. 

Beide Spieler verfolgen während der ganzen Partie das Ziel den gegnerischen Spieler Schach-Matt zu setzen. Ein Spieler ist Schach-Matt, wenn folgende Bedingungen erfüllt sind:
1. Der König wird durch eine gegnerische Figur bedroht.
2. Der König kann dieser Bedrohung nicht mehr ausweichen.

Eine solche Bedrohung liegt genau dann vor, wenn dem Spieler droht, im nächsten Zug seinen König geschlagen zu bekommen. Dies kann aber auch auf drei unterschiedliche Weisen pariert werden:
1. Der König bewegt sich aus dem "Schach" 
2. Der Spieler schlägt die Schach-gebende Figur
3. Eine Figur stellt sich zwischen die Schach-gebende Figur und den König

Ein anderer Spielausgang neben dem Schach-Matt liegt in dem Patt. Ein Patt ist dann gegeben, wenn der Spieler, der am Zug ist, keine Figur mehr ziehen kann und zusätzlich der König nicht im Schach steht.

## Besondere Spielzüge

### "En passent" schlagen

### Rochade

## Spielregeln

### Dreifache Stellungswiederholung

### 50-Züge Regeln

### Remisangebot

## Spielphasen